In [1]:
import pandas as pd
import numpy as np
import statsmodels
from sklearn.metrics import root_mean_squared_error

from scipy.stats import wilcoxon
from scipy.stats import mannwhitneyu

import pmdarima as pm
from pmdarima import auto_arima
from scipy import stats

import json
from tqdm import tqdm

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def fixed_window_split(data, window_size=12, test_size=6):
    splits = []
    for i in range(len(data) - window_size - test_size + 1):
        train_idx = list(range(i, i + window_size))
        test_idx = list(range(i + window_size, i + window_size + test_size))
        splits.append([train_idx, test_idx])
    return splits

In [4]:
def get_topic_lead(n=5, tpcs=None, mode="pos"):
    df = pd.read_csv("./data/tpc_age_all_corr.csv")
    df = df[df["drug_name"] == "syn_opioid"]

    if tpcs != None:
        df = df[df["topic"].isin(tpcs)]
        topics = list(df["topic"])
        leads = list(df["lead"])
        return topics, leads

    if mode=="pos":
        df = df[(df["trend_r"]>0.4) & (df["detrend_r"]>0.2)]
        df = df.sort_values(by=["trend_r"], ascending=False)
    else:
        df = df[(df["trend_r"]<-0.4) & (df["detrend_r"]<-0.2)]
        df = df.sort_values(by=["trend_r"], ascending=True)
    
    df = df[df["lead"]>0]
    df = df.reset_index(drop=True)

    topics = list(df["topic"])[:n]
    leads = list(df["lead"])[:n]
    # leads = [i  if i!=0 else 1 for i in leads]

    return topics, leads

def get_pvalue(x,y):
    _, pvalue = wilcoxon(x, y)
    return pvalue

In [5]:
get_topic_lead()

([198, 166, 28, 103, 123], [3, 3, 3, 3, 3])

In [6]:
deaths_df = pd.read_csv("../data/wonders_death_census_merge.csv")

drug_names = ["heroin", "nat_opioid", "methadone",  "syn_opioid", "cocaine", "unspecified", "cannabis"]
cols = ["Month Code"] + [i+"_norm" for i in drug_names]

deaths_df = deaths_df[cols]
deaths_df

,Month Code,heroin_norm,nat_opioid_norm,methadone_norm,syn_opioid_norm,cocaine_norm,unspecified_norm,cannabis_norm
0,2021/01,0.284169,0.356190,0.103060,1.670358,0.550860,0.049421,0.032545
1,2021/02,0.226655,0.324912,0.094038,1.473861,0.484958,0.034059,0.031647
2,2021/03,0.294457,0.374927,0.102773,1.879459,0.631710,0.036769,0.030440
3,2021/04,0.273307,0.382992,0.100343,1.889949,0.666545,0.039776,0.033146
4,2021/05,0.245234,0.366044,0.097310,1.855526,0.634777,0.037358,0.031935
5,2021/06,0.217469,0.346384,0.087048,1.788849,0.610841,0.039759,0.026205
6,2021/07,0.241494,0.347486,0.087022,1.829571,0.640771,0.038242,0.027100
7,2021/08,0.225393,0.344860,0.091481,1.850687,0.655414,0.035810,0.027986
8,2021/09,0.206631,0.335662,0.085419,1.798921,0.629517,0.036995,0.026769
9,2021/10,0.198417,0.307847,0.081171,1.778237,0.634634,0.029161,0.030364


In [7]:
tpc7_norm = pd.read_csv("../data/topic_norms2.csv")
# tpc7_norm = pd.read_csv("../KY/data/tpc8_my_v2.csv")
tpc7_norm

,group_id,0,1,10,100,101,102,103,104,105,...,94,95,96,97,98,99,month,month_str,year,Month Code
0,12_2020,0.000000,0.001587,0.000000,0.000000,0.000000,0.003819,0.000000,0.000000,0.000972,...,0.000939,0.000000,0.000000,0.002057,0.008617,0.000000,12,12,2020,2020/12
1,1_2021,0.001447,0.001786,0.001284,0.001770,0.001845,0.001761,0.002165,0.001946,0.003465,...,0.003066,0.001856,0.001837,0.002593,0.002081,0.001056,1,1,2021,2021/01
2,2_2021,0.002392,0.001710,0.001697,0.001766,0.001844,0.001751,0.002028,0.001719,0.002499,...,0.002365,0.001619,0.001724,0.003770,0.002006,0.001492,2,2,2021,2021/02
3,3_2021,0.001575,0.001816,0.001321,0.002326,0.002201,0.002283,0.003623,0.001679,0.007434,...,0.003712,0.001946,0.001295,0.001546,0.002030,0.001485,3,3,2021,2021/03
4,4_2021,0.001103,0.001427,0.001174,0.001730,0.002669,0.002520,0.003153,0.002088,0.002516,...,0.002221,0.001559,0.001073,0.001579,0.002358,0.001332,4,4,2021,2021/04
5,5_2021,0.000923,0.001540,0.001151,0.001752,0.002691,0.003066,0.002517,0.002172,0.002370,...,0.002225,0.001696,0.001145,0.001482,0.002016,0.001429,5,5,2021,2021/05
6,6_2021,0.001821,0.002116,0.001634,0.002161,0.001928,0.001814,0.002804,0.001946,0.003266,...,0.002322,0.001919,0.001413,0.001777,0.001754,0.001465,6,6,2021,2021/06
7,7_2021,0.001596,0.002081,0.000926,0.001691,0.001567,0.002017,0.001750,0.001897,0.002855,...,0.002823,0.002589,0.016589,0.002204,0.001415,0.001546,7,7,2021,2021/07
8,8_2021,0.001055,0.001847,0.001609,0.002242,0.002498,0.001208,0.001896,0.002098,0.004109,...,0.002936,0.002005,0.001406,0.002301,0.002971,0.001140,8,8,2021,2021/08
9,9_2021,0.001264,0.002044,0.001282,0.002044,0.002532,0.002875,0.002612,0.002194,0.003903,...,0.002845,0.001687,0.001419,0.001955,0.002209,0.001529,9,9,2021,2021/09


# Opioid Overdose Deaths

In [8]:
death_tpc7_df = pd.merge(deaths_df, tpc7_norm, on='Month Code', how='inner')

years = [2022,2023, 2024]
death_tpc7_df = death_tpc7_df[death_tpc7_df["year"].isin(years)]
death_tpc7_df = death_tpc7_df.loc[:41]
death_tpc7_df

,Month Code,heroin_norm,nat_opioid_norm,methadone_norm,syn_opioid_norm,cocaine_norm,unspecified_norm,cannabis_norm,group_id,0,...,93,94,95,96,97,98,99,month,month_str,year
12,2022/01,0.183742,0.317045,0.093973,1.789981,0.638293,0.031825,0.034226,1_2022,0.001161,...,0.002038,0.003075,0.001945,0.001093,0.001215,0.001923,0.001337,1,1,2022
13,2022/02,0.164193,0.297769,0.071741,1.740089,0.636662,0.028516,0.031218,2_2022,0.001465,...,0.002282,0.002311,0.001734,0.001652,0.002375,0.001896,0.002110,2,2,2022
14,2022/03,0.171032,0.311159,0.076214,1.893357,0.690130,0.032106,0.033906,3_2022,0.001615,...,0.002800,0.002724,0.001892,0.001560,0.002081,0.001988,0.002052,3,3,2022
15,2022/04,0.152346,0.292697,0.074974,1.762180,0.669365,0.031489,0.027890,4_2022,0.001384,...,0.002665,0.002741,0.001796,0.002060,0.001948,0.001987,0.001506,4,4,2022
16,2022/05,0.146568,0.297332,0.084224,1.802572,0.683383,0.029373,0.026376,5_2022,0.001540,...,0.002109,0.002658,0.001738,0.001352,0.001669,0.001967,0.001788,5,5,2022
17,2022/06,0.145289,0.296868,0.082081,1.784505,0.669826,0.027560,0.027859,6_2022,0.001642,...,0.002032,0.002441,0.001808,0.001472,0.001518,0.001918,0.001868,6,6,2022
18,2022/07,0.148495,0.316750,0.088319,1.903494,0.694275,0.025448,0.028741,7_2022,0.001832,...,0.002231,0.002362,0.001828,0.001457,0.001682,0.001836,0.001787,7,7,2022
19,2022/08,0.140605,0.282407,0.090645,1.891885,0.712000,0.028121,0.029617,8_2022,0.001907,...,0.001902,0.002590,0.001729,0.001179,0.001337,0.002292,0.001674,8,8,2022
20,2022/09,0.130329,0.295333,0.074132,1.799200,0.691402,0.020625,0.028397,9_2022,0.001923,...,0.001772,0.002465,0.001854,0.001245,0.001702,0.002030,0.001793,9,9,2022
21,2022/10,0.124854,0.295408,0.085128,1.925083,0.721346,0.026882,0.028077,10_2022,0.001798,...,0.001824,0.002194,0.001709,0.001274,0.001784,0.001999,0.002103,10,10,2022


In [9]:
def cdc_arima(drug_timeseries, prediction_lag=6):
    fixed_cv_generator = fixed_window_split(drug_timeseries, window_size=12, test_size=6)
    absolute_errors = []
    predictions = []
    for train_idx, test_idx in fixed_cv_generator:
        train_data = drug_timeseries.iloc[train_idx]
        test_data = drug_timeseries.iloc[test_idx]
        
        model = auto_arima(train_data, seasonal=False, error_action='ignore', suppress_warnings=True, trace=False)
        preds = model.predict(n_periods=prediction_lag)

        x = preds.iloc[prediction_lag-1]
        z = test_data.iloc[prediction_lag-1]
        ae = abs(z - x)
        
        absolute_errors.append(ae)
        predictions.append(x)
    return absolute_errors, predictions

In [10]:
def tiktok_arima(drug_timeseries, feat, prediction_lag=6):
    fixed_cv_generator = fixed_window_split(drug_timeseries, window_size=12, test_size=6)
    absolute_errors = []
    predictions = []
    # print(feat)
    for train_idx, test_idx in fixed_cv_generator:
        train_data = drug_timeseries.iloc[train_idx]
        test_data = drug_timeseries.iloc[test_idx]
        
        train_feat = feat.iloc[train_idx]
        test_feat = feat.iloc[test_idx]
        
        model = auto_arima(train_data, X = train_feat.to_frame(name='TikTok'), seasonal=False, error_action='ignore', suppress_warnings=True, trace=False)
        preds = model.predict(n_periods=prediction_lag, X = test_feat.to_frame(name='TikTok'))

        # model = auto_arima(train_data, X = train_feat, seasonal=False, error_action='ignore', suppress_warnings=True, trace=False)
        # preds = model.predict(n_periods=prediction_lag, X = test_feat)
        
        x = preds.iloc[prediction_lag-1]
        z = test_data.iloc[prediction_lag-1]
        ae = abs(z - x)
        
        absolute_errors.append(ae)
        predictions.append(x)
    return absolute_errors, predictions

In [11]:
prediction_results = {}

cdc_errors, cdc_predictions = cdc_arima(death_tpc7_df["syn_opioid_norm"])
prediction_results["cdc"] = {"pvalue":"NA", "error":cdc_errors, "predictions":cdc_predictions}

# topics, leads = get_topic_lead(n=5)
topics, leads = [166, 8], [2, 2]
for i in tqdm(range(len(topics)), desc="Running TikTok Arima"):
    topic_num = str(topics[i])
    topic_lead = leads[i]
    if topic_lead == 0: topic_lead += 1
    
    tmp_df = death_tpc7_df.copy()
    tmp_df[topic_num] = tmp_df[topic_num].shift(topic_lead)
    tmp_df = tmp_df.dropna()
    feat = tmp_df[topic_num]
    
    tiktok_errors, tiktok_predictions = tiktok_arima(tmp_df["syn_opioid_norm"], feat, prediction_lag=6)

    pvalue = get_pvalue(cdc_errors[topic_lead:], tiktok_errors)
    
    prediction_results[topic_num] = {"pvalue":pvalue, "error":tiktok_errors, "predictions":tiktok_predictions}

with open("./data/arima_results.json", 'w') as f:
    json.dump(prediction_results, f, indent=4)

Running TikTok Arima: 100%|██████████| 2/2 [00:05<00:00,  2.51s/it]


In [12]:
len(prediction_results['cdc']['predictions'])

13

In [13]:
prediction_results = {}

cdc_errors, cdc_predictions = cdc_arima(death_tpc7_df["syn_opioid_norm"])
prediction_results["cdc"] = {"pvalue":"NA", "error":cdc_errors, "predictions":cdc_predictions}

topics, leads = get_topic_lead(tpcs=[187, 149, 27])
for i in tqdm(range(len(topics)), desc="Running TikTok Arima"):
    topic_num = str(topics[i])
    topic_lead = leads[i]
    if topic_lead == 0: topic_lead += 1
    
    tmp_df = death_tpc7_df.copy()
    tmp_df[topic_num] = tmp_df[topic_num].shift(topic_lead)
    tmp_df = tmp_df.dropna()
    feat = tmp_df[topic_num]
    
    tiktok_errors, tiktok_predictions = tiktok_arima(tmp_df["syn_opioid_norm"], feat, prediction_lag=6)

    pvalue = get_pvalue(cdc_errors[topic_lead:], tiktok_errors)
    
    prediction_results[topic_num] = {"pvalue":pvalue, "error":tiktok_errors, "predictions":tiktok_predictions}

with open("./data/arima_results_NEG_CTRL.json", 'w') as f:
    json.dump(prediction_results, f, indent=4)

Running TikTok Arima: 100%|██████████| 3/3 [00:10<00:00,  3.60s/it]


In [14]:
prediction_results = {}

cdc_errors, cdc_predictions = cdc_arima(death_tpc7_df["syn_opioid_norm"])
prediction_results["cdc"] = {"pvalue":"NA", "error":cdc_errors, "predictions":cdc_predictions}

leads = list(range(1,7))
topics = [198] * len(leads)
for i in tqdm(range(len(topics)), desc="Running TikTok Arima"):
    topic_num = str(topics[i])
    topic_lead = leads[i]
    if topic_lead == 0: topic_lead += 1
    
    tmp_df = death_tpc7_df.copy()
    tmp_df[topic_num] = tmp_df[topic_num].shift(topic_lead)
    tmp_df = tmp_df.dropna()
    feat = tmp_df[topic_num]
    
    tiktok_errors, tiktok_predictions = tiktok_arima(tmp_df["syn_opioid_norm"], feat, prediction_lag=6)

    pvalue = get_pvalue(cdc_errors[topic_lead:], tiktok_errors)
    
    prediction_results[f"{topic_lead}_mon"] = {"pvalue":pvalue, "error":tiktok_errors, "predictions":tiktok_predictions}

with open("./data/arima_results_TPC198_LEAD_TIMES.json", 'w') as f:
    json.dump(prediction_results, f, indent=4)

Running TikTok Arima: 100%|██████████| 6/6 [00:19<00:00,  3.19s/it]


In [15]:
prediction_results = {}

cdc_errors, cdc_predictions = cdc_arima(death_tpc7_df["syn_opioid_norm"])
prediction_results["cdc"] = {"pvalue":"NA", "error":cdc_errors, "predictions":cdc_predictions}

leads = list(range(1,7))
topics = [166] * len(leads)
for i in tqdm(range(len(topics)), desc="Running TikTok Arima"):
    topic_num = str(topics[i])
    topic_lead = leads[i]
    if topic_lead == 0: topic_lead += 1
    
    tmp_df = death_tpc7_df.copy()
    tmp_df[topic_num] = tmp_df[topic_num].shift(topic_lead)
    tmp_df = tmp_df.dropna()
    feat = tmp_df[topic_num]
    
    tiktok_errors, tiktok_predictions = tiktok_arima(tmp_df["syn_opioid_norm"], feat, prediction_lag=6)

    pvalue = get_pvalue(cdc_errors[topic_lead:], tiktok_errors)
    
    prediction_results[f"{topic_lead}_mon"] = {"pvalue":pvalue, "error":tiktok_errors, "predictions":tiktok_predictions}

with open("./data/arima_results_TPC166_LEAD_TIMES.json", 'w') as f:
    json.dump(prediction_results, f, indent=4)

Running TikTok Arima: 100%|██████████| 6/6 [00:18<00:00,  3.10s/it]


In [17]:
prediction_results = {}

cdc_errors, cdc_predictions = cdc_arima(death_tpc7_df["syn_opioid_norm"])
prediction_results["cdc"] = {"pvalue":"NA", "error":cdc_errors, "predictions":cdc_predictions}

topics, leads = get_topic_lead(mode="neg")
for i in tqdm(range(len(topics)), desc="Running TikTok Arima"):
    topic_num = str(topics[i])
    topic_lead = leads[i]
    if topic_lead == 0: topic_lead += 1
    
    tmp_df = death_tpc7_df.copy()
    tmp_df[topic_num] = tmp_df[topic_num].shift(topic_lead)
    tmp_df = tmp_df.dropna()
    feat = tmp_df[topic_num]
    
    tiktok_errors, tiktok_predictions = tiktok_arima(tmp_df["syn_opioid_norm"], feat, prediction_lag=6)

    pvalue = get_pvalue(cdc_errors[topic_lead:], tiktok_errors)
    
    prediction_results[topic_num] = {"pvalue":pvalue, "error":tiktok_errors, "predictions":tiktok_predictions}

with open("./data/arima_results_NEG_CORR.json", 'w') as f:
    json.dump(prediction_results, f, indent=4)

Running TikTok Arima: 100%|██████████| 5/5 [00:15<00:00,  3.04s/it]


In [18]:
prediction_results.keys()

dict_keys(['cdc', '67', '138', '22', '64', '135'])

## Heart Attack  Deaths

In [19]:
heart_attack_df = pd.read_csv("../data/wonders_death_census_heart_attack.csv")

heart_attack_df = heart_attack_df[["Month Code", "heart_attack_norm"]]
heart_attack_df

,Month Code,heart_attack_norm
0,2021/01,2.710000
1,2021/02,2.309953
2,2021/03,2.388203
3,2021/04,2.212072
4,2021/05,2.142336
5,2021/06,2.101498
6,2021/07,2.173142
7,2021/08,2.259643
8,2021/09,2.247374
9,2021/10,2.309453


In [20]:
heart_attack_tpc7_df = pd.merge(heart_attack_df, tpc7_norm, on='Month Code', how='inner')

years = [2022,2023, 2024]
heart_attack_tpc7_df = heart_attack_tpc7_df[heart_attack_tpc7_df["year"].isin(years)]
heart_attack_tpc7_df = heart_attack_tpc7_df.loc[:41]
heart_attack_tpc7_df

,Month Code,heart_attack_norm,group_id,0,1,10,100,101,102,103,...,93,94,95,96,97,98,99,month,month_str,year
12,2022/01,2.827882,1_2022,0.001161,0.001777,0.001192,0.001845,0.003699,0.002745,0.002740,...,0.002038,0.003075,0.001945,0.001093,0.001215,0.001923,0.001337,1,1,2022
13,2022/02,2.272291,2_2022,0.001465,0.002167,0.001524,0.001980,0.002113,0.001962,0.002514,...,0.002282,0.002311,0.001734,0.001652,0.002375,0.001896,0.002110,2,2,2022
14,2022/03,2.208416,3_2022,0.001615,0.002501,0.001544,0.001948,0.002279,0.001919,0.002498,...,0.002800,0.002724,0.001892,0.001560,0.002081,0.001988,0.002052,3,3,2022
15,2022/04,2.072271,4_2022,0.001384,0.001962,0.001452,0.002330,0.002153,0.002339,0.003565,...,0.002665,0.002741,0.001796,0.002060,0.001948,0.001987,0.001506,4,4,2022
16,2022/05,2.101402,5_2022,0.001540,0.002030,0.001475,0.002370,0.002148,0.002258,0.003439,...,0.002109,0.002658,0.001738,0.001352,0.001669,0.001967,0.001788,5,5,2022
17,2022/06,2.028350,6_2022,0.001642,0.002123,0.001441,0.002060,0.002160,0.001928,0.002642,...,0.002032,0.002441,0.001808,0.001472,0.001518,0.001918,0.001868,6,6,2022
18,2022/07,2.119651,7_2022,0.001832,0.002067,0.001421,0.001850,0.002216,0.002068,0.002384,...,0.002231,0.002362,0.001828,0.001457,0.001682,0.001836,0.001787,7,7,2022
19,2022/08,2.036080,8_2022,0.001907,0.002220,0.001078,0.001687,0.002285,0.002004,0.002579,...,0.001902,0.002590,0.001729,0.001179,0.001337,0.002292,0.001674,8,8,2022
20,2022/09,2.030863,9_2022,0.001923,0.001909,0.001605,0.002042,0.002602,0.002360,0.002380,...,0.001772,0.002465,0.001854,0.001245,0.001702,0.002030,0.001793,9,9,2022
21,2022/10,2.167324,10_2022,0.001798,0.001996,0.001663,0.002067,0.002347,0.002391,0.002321,...,0.001824,0.002194,0.001709,0.001274,0.001784,0.001999,0.002103,10,10,2022


In [22]:
prediction_results = {}

cdc_errors, cdc_predictions = cdc_arima(heart_attack_tpc7_df["heart_attack_norm"])
prediction_results["cdc"] = {"pvalue":"NA", "error":cdc_errors, "predictions":cdc_predictions}

topics, leads = get_topic_lead(tpcs=[198,166,123])
for i in tqdm(range(len(topics)), desc="Running TikTok Arima"):
    topic_num = str(topics[i])
    topic_lead = leads[i]
    if topic_lead == 0: topic_lead += 1
    
    tmp_df = heart_attack_tpc7_df.copy()
    tmp_df[topic_num] = tmp_df[topic_num].shift(topic_lead)
    tmp_df = tmp_df.dropna()
    feat = tmp_df[topic_num]
    
    tiktok_errors, tiktok_predictions = tiktok_arima(tmp_df["heart_attack_norm"], feat, prediction_lag=6)

    pvalue = get_pvalue(cdc_errors[topic_lead:], tiktok_errors)
    
    prediction_results[topic_num] = {"pvalue":pvalue, "error":tiktok_errors, "predictions":tiktok_predictions}

with open("./data/arima_results_HEART_ATTACK.json", 'w') as f:
    json.dump(prediction_results, f, indent=4)

Running TikTok Arima: 100%|██████████| 3/3 [00:12<00:00,  4.15s/it]


In [66]:
np.mean(prediction_results["cdc"]["error"])

0.19055807639746508

In [69]:
np.mean(prediction_results["166"]["pvalue"])

0.556640625